## Plain-text Generation

In [ ]:
a = ['f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u']
text = []
z = 8
for x in range(z):
    temporary = []
    for i in range(8):
        for j in range(16):
            x1 = a[i] + a[j]
            x2 = 'ff'*x + 'ff'*(7-x)
            sting = x1 + x2
            temporary.append(sting)
    text.append(temporary)

file = open('plaintexts.txt','w')
for i in text:
    sting = ' '.join(i) + '\n'
    file.write(sting)
file.close()

## Server code to generate Output-Input pair

This pexpect module is not compatible with windows. So, run this notebook in mac or linux envirnoment

In [ ]:
import pexpect 

child = pexpect.spawn('/usr/bin/ssh student@172.27.26.188')
def calls():
    
    child.expect('student@172.27.26.188\'s password:')
    child.sendline('cs641')  # Entering the password
   
    child.expect('Enter your group name: ', timeout=1)
    child.sendline("team_ethereum")  # Entering the group name

    child.expect('Enter password: ', timeout=1)  # Waiting for the password prompt
    child.sendline("team_ethereum")  # Entering the password

    child.expect('\r\n\r\n\r\nYou have solved 4 levels so far.\r\nLevel you want to start at: ',
                timeout=1)  
    child.sendline("5") 

    child.expect('.*') 
    child.sendline("go")  

    child.expect('.*') 
    child.sendline("wave") 

    child.expect('.*')  
    child.sendline("dive")  

    child.expect('.*')  
    child.sendline("go")  

    child.expect('.*')  
    child.sendline("read")  

    child.expect('.*')  
    child.sendline("password") 

    child.expect('.*')  
    child.sendline("c")  

    child.expect('.*')  

calls()


f = open("plaintexts.txt", 'r')  
f1 = open("ciphertexts1.txt", 'w')  


for line in f.readlines():
    li = line.split()

    for l in li:
        child.sendline(l)

        f1.write(s)
        f1.write(" ")
        child.expect("Slowly, a new text starts*")
        s = str(child.before)[48:64]

        f1.write(s)
        f1.write(" ")
        child.sendline("c")
        child.expect('The text in the screen vanishes!')
    f1.write("\n")

child.sendline("ffffffffffffffmu")
s = str(child.before)[48:64]
f1.write(s)
f1.write(" ")

child.close()


f.close()
f1.close()


## Analysis

In [8]:
import numpy as np
from pyfinite import ffield
import galois

F = ffield.FField(7, gen=0x83, useLUT=-1)

def expo(base,power):
    ans = base
    for i in range(1,power):
        ans = F.Multiply(ans,base)
    return ans


def Linear(linmat,msg):
    ans = [0]*8
    for i in range(8):
        temp  = []
        mul = [F.Multiply(linmat[i][j],msg[i]) for j in range(8)]
        for k in range(8):
            temp.append(np.bitwise_xor(ans[k],mul[k]))
        ans = temp
    return ans



def decode_block(cipher):
  plain= ""
  for i in range(0,len(cipher),2):
      plain+=chr(16*(ord(cipher[i:i+2][0]) - ord('f')) + ord(cipher[i:i+2][1]) - ord('f'))
  return plain



PosExpo = [[] for i in range(8)]   
posDiaVals=[[[] for i in range(8)] for j in range(8)]
input_file = open('plaintexts.txt','r')
output_file = open('ciphertexts.txt','r')
input = (input_file.readlines()[0]).strip().split(' ')
output = output_file.readlines()

in_string = []
for msg in input:
    in_string.append(decode_block(msg)[0])

out_string = []
for i in range(len(output)):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg)[i])
    out_string.append(x)

for k in range(8):
    for i in range(1, 127):
        for j in range(1, 128):
          flag = True
          for m in range(128):
            if(ord(out_string[k][m]) != expo(F.Multiply(expo(F.Multiply(expo(ord(in_string[m]), i), j), i), j), i)):
              flag = False
              break
          if(flag):
            PosExpo[k].append(i)
            posDiaVals[k][k].append(j)
print("Possible diagonal values: \n")
print(posDiaVals)
print("\n\nPossible exponents: \n")
print(PosExpo)


out_string = []
for i in range(len(output)-1):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg)[i+1])
    out_string.append(x)

for ind in range(7):
  for i in range(1, 128):
      for p1, e1 in zip(PosExpo[ind+1], posDiaVals[ind+1][ind+1]):
          for p2, e2 in zip(PosExpo[ind], posDiaVals[ind][ind]):
              for k in range(128):
                  flag = True
                  x1 = F.Multiply(expo(F.Multiply(expo(ord(in_string[k]), p2), e2), p2), i)
                  x2 = F.Multiply(expo(F.Multiply(expo(ord(in_string[k]), p2), i), p1), e1)
                  c1 = np.bitwise_xor(x1,x2)
                  if(ord(out_string[ind][k]) != expo(c1,p1)):
                      flag = False
                      break
              if flag:
                  PosExpo[ind+1] = [p1]
                  posDiaVals[ind+1][ind+1] = [e1]
                  PosExpo[ind] = [p2]
                  posDiaVals[ind][ind] = [e2]
                  posDiaVals[ind][ind+1] = [i]

print('\n\n*******************************************************88\n\n')
print("Linear Transformation Matrix A values: \n")
print(posDiaVals)
print("\n\nExponent Vector E values : \n")
print(PosExpo)



def EAEAE (msg, lin_mat, exp_mat):
  msg = [ord(m) for m in msg]
  res = [expo(msg[i], exp_mat[i]) for i in range(8)]
  res = Linear(lin_mat, res)
  res = [expo(res[i], exp_mat[i]) for i in range(8)]
  res = Linear(lin_mat, res)
  res = [expo(res[i], exp_mat[i]) for i in range(8)]
  return res


input_file = open('plaintexts.txt','r')
output_file = open('ciphertexts.txt','r')
input = input_file.readlines()
output = output_file.readlines()


in_string = []
for i in range(len(input)):
    x = []
    for msg in input[i].strip().split(' '):
        x.append(decode_block(msg))
    in_string.append(x)


out_string = []
for i in range(len(output)):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg))
    out_string.append(x)

for indexex in range(0,6):
    offset = indexex + 2
    
    exp_list = [e[0] for e in PosExpo]
    lin_tran_lst = np.zeros((8,8),int)

    for i in range(8):
        for j in range(8):     
            if(len(posDiaVals[i][j]) != 0):
                lin_tran_lst[i][j] = posDiaVals[i][j][0]
            else:
                lin_tran_lst[i][j] = 0
    
    for index in range(8):
        if(index > (7-offset)):
            continue

        for i in range(127):
            lin_tran_lst[index][index+offset] = i+1
            flag = True
            for inps, outs in zip(in_string[index], out_string[index]):
                x1 = EAEAE(inps, lin_tran_lst, exp_list)[index+offset]
                x2 = outs[index+offset]
                if x1 != ord(x2):
                    flag = False
                    break
            if flag==True:
                posDiaVals[index][index+offset] = [i+1]
                
A = np.zeros((8,8),dtype='int')

for i in range(0,8):
    for j in range(0,8):
        if len(posDiaVals[j][i]) != 0:
            A[i][j] = posDiaVals[j][i][0]



E = exp_list

print('\n\nLinear Transformation Matrix :\n',A)
print('\n\n')
print('Exponent Vector : \n',E)


Einverse = np.zeros((128, 128), dtype = int)

for base in range(0,128):
    temp = 1
    for power in range(1,127):
        result = F.Multiply(temp, base)
        Einverse[power][result] = base
        temp = result
    
GF = galois.GF(2**7)
A = GF(A)
invA = np.linalg.inv(A)



password = "lhkhkumjkujmhhirmmflltlrkukfmmll"
GF = galois.GF(2**7)

def Einv(block, E):
    return [Einverse[E[i]][block[i]] for i in range(8) ]

def Ainv(block, A):
    block = GF(block)
    A = GF(A)
    return np.matmul(A,block)

decrypted_password = ""
for i in range(0,2): 
    elements = password[16*i:16*(i+1)]
    currentBlock = []
    for j in range(0,15,2):
        currentBlock+=[(ord(elements[j]) - ord('f'))*16 + (ord(elements[j+1]) - ord('f'))]
    EAEAE = Einv(Ainv(Einv(Ainv(Einv(currentBlock, E), invA),E), invA),E)
    for ch in EAEAE:
        decrypted_password += chr(ch)
    
print("\n\nPassword is",decrypted_password)

Possible diagonal values: 

[[[73, 84, 20], [], [], [], [], [], [], []], [[], [29, 52, 70], [], [], [], [], [], []], [[], [], [119, 43, 5], [], [], [], [], []], [[], [], [], [37, 126, 12], [], [], [], []], [[], [], [], [], [109, 112, 67], [], [], []], [[], [], [], [], [], [11, 106, 70], [], []], [[], [], [], [], [], [], [27, 14], []], [[], [], [], [], [], [], [], [38, 8, 124]]]


Possible exponents: 

[[18, 21, 88], [6, 7, 114], [2, 38, 87], [22, 37, 68], [59, 90, 105], [40, 89, 125], [20, 108], [23, 48, 56]]




Linear Transformation Matrix A values: 

[[[84], [113], [], [], [], [], [], []], [[], [70], [30], [], [], [], [], []], [[], [], [43], [12], [], [], [], []], [[], [], [], [12], [111], [], [], []], [[], [], [], [], [112], [98], [], []], [[], [], [], [], [], [11], [94], []], [[], [], [], [], [], [], [27], [11]], [[], [], [], [], [], [], [], [38]]]


Exponent Vector E values : 

[[21], [114], [38], [68], [90], [40], [20], [23]]


Linear Transformation Matrix :
 [[ 84   0   0   0  